# Classificateur CNN

Dans ce tutoriel, nous allons designer un réseau à convolution pour faire de la classification d'images.

In [4]:
import math
import torch
import numpy as np
from torch import optim, nn
from torch.autograd import Variable
from torchvision import transforms
import torchvision.models as models
from torchvision.datasets.cifar import CIFAR100
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F

from pytoune.framework import Model, ModelCheckpoint, Callback, CSVLogger, EarlyStopping, ReduceLROnPlateau
from pytoune import torch_to_numpy
from pytoune.layers import Flatten
from torchvision.utils import make_grid

torch.manual_seed(42)
np.random.seed(42)

In [5]:
# Training hyperparameters
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")
batch_size = 32
learning_rate = 0.01
n_epoch = 5
num_classes = 100

In [6]:
device

device(type='cuda', index=0)

In [7]:
def load_cifar100(download=False, path='./data', transform=None):
    """Loads the cifar100 dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train and test dataset
    """
    train_dataset = CIFAR100(path, train=True, download=download, transform=transform)
    test_dataset = CIFAR100(path, train=False, download=download, transform=transform)
    return train_dataset, test_dataset


def load_cifar100_with_validation_set(download=False, path='./data', train_split=0.8):
    """Loads the CIFAR10 dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train, valid and test dataset ready to be ingest in a neural network
    """
    norm_coefs = {}
    norm_coefs['imagenet'] = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(*norm_coefs['imagenet'])
    ])
    train, test = load_cifar100(download, path, transform=transform)
    lengths = [round(train_split*len(train)), round((1.0-train_split)*len(train))]
    train, valid = random_split(train, lengths)
    return train, valid, test
  
def count_number_of_parameters(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

In [5]:
train, valid, test = load_cifar100_with_validation_set(download=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
len(train), len(valid), len(test)

(40000, 10000, 10000)

In [7]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

In [8]:
def train(name, pytorch_module, params=None):
    if not params:
      params = pytorch_module.parameters()

    optimizer = optim.SGD(params, lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    
    early_stopping = EarlyStopping(patience=5)
    lr_scheduler = ReduceLROnPlateau(patience=2)
    callbacks = [early_stopping, lr_scheduler]

    # Pytoune Model
    model = Model(pytorch_module, optimizer, loss_function, metrics=['accuracy'])

    # Send model on GPU
    model.to(device)

    # Train
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch)
    return model

In [9]:
# Define our convolutional model
class CifarNet(nn.Module):
    def __init__(self):
        super(CifarNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 10, 3, padding=1)
        self.conv2 = nn.Conv2d(10, 50, 3, padding=1)
        self.conv3 = nn.Conv2d(50, 150, 3, padding=1)
        
        self.batchnorm1 = nn.BatchNorm2d(10)
        self.batchnorm2 = nn.BatchNorm2d(50)
        self.batchnorm3 = nn.BatchNorm2d(150)
        
        self.fc1 = nn.Linear(150 * 56 * 56, num_classes)

    def forward(self, x):
        # x shape: (batch_size, 3, 224, 224)
        x = F.max_pool2d(F.relu(self.batchnorm1(self.conv1(x))), (2, 2))
        # x shape: (batch_size, 10, 112, 112)
        x = F.max_pool2d(F.relu(self.batchnorm2(self.conv2(x))), (2, 2))
        # x shape: (batch_size, 50, 56, 56)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        # x shape: (batch_size, 150, 56, 56)
        x = x.view(x.shape[0], -1)
        # x shape: (batch_size, 470400)
        x = self.fc1(x)
        return x


In [10]:
net = CifarNet()
model = train('simple_cnn', net)

Epoch 1/5 118.37s Step 1250/1250: loss: 4.469712, acc: 11.855000, val_loss: 3.543773, val_acc: 17.420000
Epoch 2/5 165.64s Step 1250/1250: loss: 3.249558, acc: 23.327500, val_loss: 3.229298, val_acc: 24.000000
Epoch 3/5 190.28s Step 1250/1250: loss: 2.748987, acc: 33.047500, val_loss: 3.138871, val_acc: 26.580000
Epoch 4/5 188.58s Step 1250/1250: loss: 2.174158, acc: 45.277500, val_loss: 3.294137, val_acc: 26.450000
Epoch 5/5 195.99s Step 1250/1250: loss: 1.577077, acc: 59.297500, val_loss: 3.471613, val_acc: 26.730000


In [11]:
model.evaluate_generator(test_loader)

(3.442967367553711, 26.25)

In [12]:
count_number_of_parameters(net)

47113000

In [15]:
from collections import OrderedDict

# Same model as above using the sequential module of Pytorch
model = nn.Sequential(
    OrderedDict([
        ('conv1', nn.Conv2d(3, 10, 3, padding=1)),
        ('bn1', nn.BatchNorm2d(10)),
        ('relu1', nn.ReLU()),
        ('maxpool1', nn.MaxPool2d(2)),
        ('conv2', nn.Conv2d(10, 50, 3, padding=1)),
        ('bn1', nn.BatchNorm2d(50)),
        ('relu2', nn.ReLU()),
        ('maxpool2', nn.MaxPool2d(2)),
        ('conv3', nn.Conv2d(50, 150, 3, padding=1)),
        ('bn1', nn.BatchNorm2d(150)),
        ('relu3', nn.ReLU()),
        ('fc1', nn.Linear(150 * 56 * 56, num_classes))
    ])
)